In [ ]:
### a metade dos links da empresa escolhida para organização em uma futura adaptação, no qual ficaria dinâmico a empresa que fosse realizar o scraping

link1 = 'faculdade-projecao/lista-reclamacoes/'
link2 = 'faculdade-projecao/lista-reclamacoes/?pagina=2'
link3 = 'faculdade-projecao/lista-reclamacoes/?pagina=3'

In [ ]:
### Importações necessárias

import time
from dotenv import load_dotenv
import os
import pyautogui as pg
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from botcity.maestro import *
from botcity.core import DesktopBot
from botcity.web import WebBot, Browser, By
import requests as rq

In [ ]:
### Utilizando a biblioteca pyautogui para abrir uma janela com 4 seleções: Página 1, Página 2, Página 3 e Cancelar

pagina_scraping = pg.confirm(
    title='Inicio do Scraping', 
    text='Escolha uma opção a seguir:',
    buttons=['Página 1','Página 2', 'Página 3','Cancelar']    
)

match pagina_scraping:
    case 'Página 1':
        url_scraping = f'https://www.reclameaqui.com.br/empresa/{link1}'
    case 'Página 2':
        url_scraping = f'https://www.reclameaqui.com.br/empresa/{link2}'
    case 'Página 3':
        url_scraping = f'https://www.reclameaqui.com.br/empresa/{link3}'


In [ ]:
# Inicia a aplicação da biblioteca Botcity

botweb = WebBot()
botweb.headless = True # True deixa a janela do navegador escondida durante o Scraping e False habilita
botweb.browser = Browser.CHROME # Escolhe qual navegador deveja utilizar para o scraping
botweb.driver_path = "chromedriver.exe" # Utiliza o chromedriver local, no qual já está habilitado dentro da pasta. Caso venha ter problemas com o chromedriver, tem que verificar qual a versão do seu google chrome e atualizar o arquivo do chromedriver

# Cria as variáveis do tipo lista
titulo_reclamacoes = []
descricao_reclamacoes = []
link_reclamacoes = []
resposta_reclamacoes = []
hora_reclamacoes = []

# Cria a função extrair_informacoes com os dados necessários para a extração
def extrair_informacoes(xpath, atributo, lista_destino):
    elementos = botweb.find_elements(xpath, by=By.XPATH)
    for elemento in elementos:
        lista_destino.append(elemento.get_attribute(atributo))

botweb.browse(url_scraping) # Abre o site estabelecido no campo de código anterior
time.sleep(2) # Aguarda 2 segundos
print("Iniciando o scraping")
    
# Faz o scraping de cada informação de acordo com a variável que foi criada
try:
    extrair_informacoes('//h4[@class="sc-1pe7b5t-1 bVKmkO"]', 'textContent', titulo_reclamacoes)
    extrair_informacoes('//p[@class="sc-1pe7b5t-2 eHoNfA"]', 'textContent', descricao_reclamacoes)
    extrair_informacoes('//div[@class="sc-1pe7b5t-0 eJgBOc"]//a', 'href', link_reclamacoes)
    extrair_informacoes('//div[@class="sc-1pe7b5t-3 gkhqVK"]//span[1]', 'textContent', resposta_reclamacoes)
    extrair_informacoes('//div[@class="sc-1pe7b5t-3 gkhqVK"]//span[2]', 'textContent', hora_reclamacoes)

except Exception as e:
    print(f"Houve um erro em {e}")
    botweb.close_page()

finally:
    print("Scraping finalizado")
    botweb.close_page()

Iniciando o scraping
Scraping finalizado


In [18]:
df = pd.DataFrame({
    'titulo_reclamacao': titulo_reclamacoes,
    'decricao': descricao_reclamacoes,
    'links': link_reclamacoes,
    'resposta_empresa': resposta_reclamacoes,
    'hora_reclamacao': hora_reclamacoes    
})

json_data = df.to_json(orient='records', indent=1)
json_data = json_data.replace("\\/","/")

In [19]:
data_to_send = {
    "dados_gerais": json_data,
    "whatsapp":"sim"
}

load_dotenv()
url = os.getenv('url_wh')

response = rq.post(url, data=data_to_send)

print("Status code: ", response.status_code)
print("Resposta do Servidor: ", response.json())

Status code:  200
Resposta do Servidor:  {'message': 'Workflow was started'}
